# DTW Example

This notebook demonstrates the DTW computation following the example described [here]().
* The first part of the notebook goes through the DTW algorithm based on dynamic programming.
* The second part of the notebook describes the usage of functions wrapped up in `dtw.py`.


## First Part 

First, define the two time series A and B

In [1]:
import numpy as np 

In [2]:
A = np.array([3,2,2,3,5,5,6])
B = np.array([1,3,2,2,3,5])

Initialize a dtw matrix based on the length of A and B, i.e., length of B define the number of rows, and length of A define the number of columns.

In [3]:
dtw_mat = np.zeros((len(B), len(A)))
print(dtw_mat)

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


Now, we need to loop through all element in the dtw matrix.

In [4]:
# define the absolute distance function
d = lambda x, y: np.abs(x - y)

for i in range(len(B)):
    for j in range(len(A)):
        if i == 0 and j == 0:
            dtw_mat[i, j] = d(B[i], A[j])
        else:
            if i == 0 and j > 0:
                choice = dtw_mat[i, j-1]
            elif i > 0 and j == 0:
                choice = dtw_mat[i-1, j]
            else:
                choice = [dtw_mat[i-1, j], dtw_mat[i, j-1], dtw_mat[i-1, j-1]]
            
            dtw_mat[i, j] = d(B[i], A[j]) + np.min(choice)

print(dtw_mat)

[[ 2.  3.  4.  6. 10. 14. 19.]
 [ 2.  3.  4.  4.  6.  8. 11.]
 [ 3.  2.  2.  3.  6.  9. 12.]
 [ 4.  2.  2.  3.  6.  9. 13.]
 [ 4.  3.  3.  2.  4.  6.  9.]
 [ 6.  6.  6.  4.  2.  2.  3.]]


We can find the warping path by backtracking.

In [5]:
path = [[len(B)-1, len(A)-1]]
while(True):
    print(path)
    i, j = path[-1][0], path[-1][1]
    if i == 0 and j == 0:
        break
    elif i == 0 and j > 0:
        path.append([i, j-1])
    elif i > 0 and j == 0:
        path.append([i-1, j])
    else:
        choice = [dtw_mat[i-1, j], dtw_mat[i, j-1], dtw_mat[i-1, j-1]]
        ind = [[i-1, j], [i, j-1], [i-1, j-1]]
        k = np.argmin(choice)
        path.append(ind[k])

[[5, 6]]
[[5, 6], [5, 5]]
[[5, 6], [5, 5], [5, 4]]
[[5, 6], [5, 5], [5, 4], [4, 3]]
[[5, 6], [5, 5], [5, 4], [4, 3], [3, 2]]
[[5, 6], [5, 5], [5, 4], [4, 3], [3, 2], [2, 2]]
[[5, 6], [5, 5], [5, 4], [4, 3], [3, 2], [2, 2], [2, 1]]
[[5, 6], [5, 5], [5, 4], [4, 3], [3, 2], [2, 2], [2, 1], [1, 0]]
[[5, 6], [5, 5], [5, 4], [4, 3], [3, 2], [2, 2], [2, 1], [1, 0], [0, 0]]


In [6]:
warp = np.zeros((len(B), len(A)))
for p in path:
    warp[p[0], p[1]] = 1 

print(warp)

[[1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1.]]


Then, we can calculate the normalized distance between time series A and time series B. 
Assume that all the point shares equal weight, i.e., $w_s=1$ for all $P_s$, then $D(A,B)$ can be computed as follows:

$$
D(A,B) = \frac{1}{k} \sum_{s = 1}^k d(P_s)
$$

In [7]:
D = np.sum(warp * dtw_mat)/len(path)
print(f'Normalized Distance: {D:2f}')

Normalized Distance: 2.111111


## Second Part

The dtw computation above is wrapped up into three functions, i.e., `computed_dtwMat`, `get_warpingPath`, `normalized_dist`. These functions are saved in the script `dtw.py`.

In [8]:
from dtw import computed_dtwMat, get_warpingPath, normalized_dist

In [9]:
dtw_mat = computed_dtwMat(A, B)
print(dtw_mat)

[[ 2.  3.  4.  6. 10. 14. 19.]
 [ 2.  3.  4.  4.  6.  8. 11.]
 [ 3.  2.  2.  3.  6.  9. 12.]
 [ 4.  2.  2.  3.  6.  9. 13.]
 [ 4.  3.  3.  2.  4.  6.  9.]
 [ 6.  6.  6.  4.  2.  2.  3.]]


In [10]:
path, warp, dtw_matrix = get_warpingPath(A, B)
print(warp)
print(dtw_mat)

[[1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1.]]
[[ 2.  3.  4.  6. 10. 14. 19.]
 [ 2.  3.  4.  4.  6.  8. 11.]
 [ 3.  2.  2.  3.  6.  9. 12.]
 [ 4.  2.  2.  3.  6.  9. 13.]
 [ 4.  3.  3.  2.  4.  6.  9.]
 [ 6.  6.  6.  4.  2.  2.  3.]]


In [11]:
D = normalized_dist(A, B)
print(f'Normalized Distance: {D:2f}')

Normalized Distance: 2.111111
